## Imports

In [1]:
import googlemaps
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob

## Read-in

In [2]:
# Get today's date
today = datetime.now()

# Get the date one week ago
one_week_ago = today - timedelta(days=7)

# Function to format the date with the correct ordinal suffix
def format_date_with_ordinal(date):
    day = date.day
    if 4 <= day <= 20 or 24 <= day <= 30:
        suffix = "th"
    else:
        suffix = ["st", "nd", "rd"][day % 10 - 1]
    return date.strftime(f"%b. {day}{suffix}")

# Format both dates
week_ending = format_date_with_ordinal(today)
week_starting = format_date_with_ordinal(one_week_ago)

print(week_starting, week_ending)

Jun. 17th Jun. 24th


In [3]:
filename = glob.glob('redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [4]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [5]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [6]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [8]:
# # Filter dates between. If only using data looking back one week, this shouldn't change the df
# df = df[(df['SOLD DATE'] >= '2024-06-10') & (df['SOLD DATE'] <= '2024-06-17')]

In [7]:
df.sort_values(ascending=False, by='SOLD DATE')

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
43,PAST SALE,2024-06-24,Condo/Co-op,10968 W 33 Ln #10968,Hialeah,FL,33018.0,569900.0,3.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Hialeah/10968-W-33rd...,MARMLS,A11502461,N,Y,25.922279,-80.352912
113,PAST SALE,2024-06-24,Condo/Co-op,20400 W Country Club Dr #512,Aventura,FL,33180.0,290000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/20400-W-Cou...,MARMLS,A11523376,N,Y,25.965848,-80.140064
23,PAST SALE,2024-06-24,Condo/Co-op,6070 W 19th Ave #309,Hialeah,FL,33012.0,200500.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Hialeah/6070-W-19th-...,MARMLS,A11578185,N,Y,25.878090,-80.321752
2,PAST SALE,2024-06-23,Condo/Co-op,2730 NE 4th St #106,Homestead,FL,33033.0,310000.0,3.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Homestead/2730-NE-4t...,Beaches MLS,F10447322,N,Y,25.473871,-80.437073
134,PAST SALE,2024-06-21,Condo/Co-op,6495 W 27th Ave Unit 41-101,Hialeah,FL,33016.0,321000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Hialeah/6495-W-27th-...,MARMLS,A11488027,N,Y,25.881449,-80.336916
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102,PAST SALE,2024-06-17,Condo/Co-op,4854 NW 7th St #506,West Miami,FL,33126.0,335000.0,3.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/4854-NW-7th-St...,MARMLS,A11529686,N,Y,25.778196,-80.275145
109,PAST SALE,2024-06-17,Condo/Co-op,15201 NE 6th Ave Unit C201,Miami,FL,33162.0,165000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/15201-NE-6th-A...,MARMLS,A11493095,N,Y,25.915314,-80.186602
111,PAST SALE,2024-06-17,Condo/Co-op,5600 Collins Ave Unit 10K,Miami Beach,FL,33140.0,557750.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/5600-Col...,MARMLS,A11503664,N,Y,25.835664,-80.121354
114,PAST SALE,2024-06-17,Condo/Co-op,15811 Collins Ave #2005,Sunny Isles Beach,FL,33160.0,1140000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/North-Miami-Beach/15...,MARMLS,A11496929,N,Y,25.922834,-80.121864


In [8]:
df = df.sort_values(by='PRICE',ascending=False)
df_top_ten = df.head(10)

## Current Week's Values

In [36]:
# Number of condos sold
current_week_condo_sales = len(df)
print(len(df))

147


In [37]:
# Average sale price
average_sale_price = round(df['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$1,167,993


In [38]:
# Average PSF
average_price_psf = round(df['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$598


In [39]:
# Total sales dollar volume
current_week_condo_sales_total = df['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$171,694,950


In [54]:
# # Define the date range
# start_date = '2024-05-20'
# end_date = '2024-05-26'

# # Filter rows where 'SOLD DATE' is within the specified range
# second_week_df = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

## Previous Week's Values

In [15]:
print('Input Previous Week Condo Sales Total (number units sold):')
previous_week_condo_sales = input()
print('Input Previous Week Condo Average Sales Price:')
previous_week_average_sales_price = input()
print('Input Previous Week Condo Average PSF:')
previous_week_average_psf = input()
print('Input Previous Week Condo Sales total (ex: 198_000_000)')
previous_week_condo_sales_total = input()

Input Previous Week Condo Sales Total (number units sold):
133
Input Previous Week Condo Average Sales Price:
852000
Input Previous Week Condo Average PSF:
569
Input Previous Week Condo Sales total (ex: 198_000_000)
169000000


## Clean Data

In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [16]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [17]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [18]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'RANK', 'COLOR'],
      dtype='object')

In [19]:
pd.set_option('display.max_columns',None)

In [20]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,RANK,COLOR
115,PAST SALE,2024-06-20,Condo/Co-op,7001 Fisher Island Dr Ph 1,Miami Beach,FL,33109.0,37000000.0,5.0,5.5,PALAZZO DEL SOL/DELLA LUN,7000.0,NaN,2016.0,NaN,5286.0,14120.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/7001-Fis...,MARMLS,A11507600,N,Y,25.762705,-80.147903,1,orange


In [21]:
def popup_html(row):
    building_name = row['LOCATION']
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(building_name) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [22]:
from folium.plugins import MarkerCluster

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'{week_starting} - {week_ending}')


### Create map container ###
m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df_top_ten.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [23]:
m.save('index.html')

## Data snagger

In [24]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [25]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,RANK,COLOR
115,PAST SALE,2024-06-20,Condo/Co-op,7001 Fisher Island Dr Ph 1,Miami Beach,FL,33109.0,37000000.0,5.0,5.5,PALAZZO DEL SOL/DELLA LUN,7000.0,NaN,2016.0,NaN,5286.0,14120.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/7001-Fis...,MARMLS,A11507600,N,Y,25.762705,-80.147903,1,orange


In [ ]:
# , {df_top_ten[df_top_ten['PRICE'].idxmax()]['ADDRESS']} | Price ${df_top_ten[df_top_ten['PRICE'].idxmax()]['PRICE']:,.0f}

In [26]:
df_top_ten['PRICE'].iloc[0]

37000000.0

In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [27]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/condo_sales_week_ending_06242024


# CREATE TEMPLATE 

In [28]:
muni_set = set(df_top_ten['CITY'])

In [29]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [30]:
df_top_ten.reset_index(inplace=True,drop=True)

In [31]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [32]:
top_sale

'PALAZZO DEL SOL/DELLA LUN at 7001 Fisher Island Dr Ph 1 in Miami Beach'

In [40]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [111]:
# df_top_ten

## RESUME HERE

In [41]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops Miami-Dade weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f}M to ${df_top_ten.at[0,'PRICE']:,.0f}M
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m Miami-Dade County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m Miami-Dade County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Adam Farence
\033[1mRESEARCH:\033[0m 
\033[1mSocial:\033[0m #MiamiDade condo sales XXXXX
\033[1mART:\033[0m

Interactive map

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, Miami-Dade County, weekly condo sales, {', '.join(str(x) for x in muni_set)}

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

Miami-Dade County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} sales average from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: PALAZZO DEL SOL/DELLA LUN closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $3,450,000M to $37,000,000M
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Adam Farence
RESEARCH: 
Social: #MiamiDade condo sales XXXXX
ART:

Interactive map

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade County, weekly condo sales, Miami Beach, Sunny Isles Beach, Bal Harbour, Miami

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 147 condo sales totaling $171,694,950 million from Jun. 17th to Jun. 24th. The previous week, brokers closed 133 condo sales totaling $169,000,000.

Last week’s units sold for an average of $1,167,993, higher than the $852,000 sales average from the previous week. The av

In [42]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [43]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [44]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf")

print(story_checklist)


HED: PALAZZO DEL SOL/DELLA LUN closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $3,450,000M to $37,000,000M
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Adam Farence
RESEARCH: 
Social: #MiamiDade condo sales XXXXX
ART:

Interactive map

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade County, weekly condo sales, Miami Beach, Sunny Isles Beach, Bal Harbour, Miami

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 147 condo sales totaling $171,694,950 million from Jun. 17th to Jun. 24th. The previous week, brokers closed 133 condo sales totaling $169,000,000.

Last week’s units sold for an average of $1,167,993, higher than the $852,000 sales average from the previous week. The av

In [59]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Bal-Harbour/10203-Collins-Ave-33154/unit-2001/home/56745771


In [53]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Miami-Beach/3917-N-Meridian-Ave-33140/unit-204/home/43402575


In [55]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Miami/1000-Brickell-Plz-33131/unit-5504/home/79135667


In [56]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Miami-Beach/7001-Fisher-Island-Dr-33109/unit-1/home/63044975


In [61]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Bal-Harbour/10225-Collins-Ave-33154/unit-402/home/43316215


In [58]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-06-20,Condo/Co-op,7001 Fisher Island Dr Ph 1,Miami Beach,FL,33109.0,37000000.0,5.0,5.5,PALAZZO DEL SOL/DELLA LUN,7000.0,NaN,2016.0,NaN,5286.0,14120.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/7001-Fis...,MARMLS,A11507600,N,Y,25.762705,-80.147903,1,orange,PALAZZO DEL SOL/DELLA LUN at 7001 Fisher Islan...
1,PAST SALE,2024-06-21,Condo/Co-op,10203 Collins Ave #2001,Bal Harbour,FL,33154.0,17500000.0,4.0,6.0,OCEANA BAL HARBOUR CONDO,7442.0,NaN,2016.0,NaN,2352.0,12339.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/10203-Co...,MARMLS,A11460429,N,Y,25.895557,-80.122903,2,blue,OCEANA BAL HARBOUR CONDO at 10203 Collins Ave ...
2,PAST SALE,2024-06-18,Condo/Co-op,17975 Collins Ave #1602,Sunny Isles Beach,FL,33160.0,9300000.0,4.0,4.5,ESTATES AT ACQUALINA,4385.0,NaN,2023.0,NaN,2121.0,8589.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11572509,N,Y,25.942332,-80.120166,3,blue,ESTATES AT ACQUALINA at 17975 Collins Ave #160...
3,PAST SALE,2024-06-18,Condo/Co-op,17749 Collins Ave #801,Sunny Isles Beach,FL,33160.0,6800000.0,3.0,6.5,17749 COLLINS AVENUE COND,4609.0,NaN,2015.0,NaN,1475.0,10288.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11516368,N,Y,25.940065,-80.120264,4,blue,17749 COLLINS AVENUE COND at 17749 Collins Ave...
4,PAST SALE,2024-06-17,Condo/Co-op,1000 S Pointe Dr #3802,Miami Beach,FL,33139.0,5900000.0,3.0,3.5,MURANO AT PORTOFINO,2618.0,NaN,2002.0,NaN,2254.0,6090.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/1000-S-P...,MARMLS,A11489764,N,Y,25.768527,-80.137591,5,blue,MURANO AT PORTOFINO at 1000 S Pointe Dr #3802 ...
5,PAST SALE,2024-06-18,Condo/Co-op,18555 Collins Ave #2005,Sunny Isles Beach,FL,33160.0,4950000.0,3.0,4.5,18555 COLLINS AVENUE COND,3415.0,NaN,2016.0,NaN,1449.0,6252.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A11368753,N,Y,25.948195,-80.120410,6,blue,18555 COLLINS AVENUE COND at 18555 Collins Ave...
6,PAST SALE,2024-06-20,Condo/Co-op,10225 Collins Ave #402,Bal Harbour,FL,33154.0,4300000.0,3.0,4.5,BELLINI CONDO,3820.0,NaN,2004.0,NaN,1126.0,5396.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/10225-Co...,MARMLS,A11561727,N,Y,25.896548,-80.123085,7,blue,BELLINI CONDO at 10225 Collins Ave #402 in Bal...
7,PAST SALE,2024-06-21,Condo/Co-op,1445 16th St Ph -3,Miami Beach,FL,33139.0,3900000.0,2.0,2.5,CAPRI SOUTH BEACH CONDO,1919.0,NaN,2008.0,NaN,2032.0,2949.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/1445-16t...,MARMLS,A11545660,N,Y,25.789262,-80.144721,8,blue,CAPRI SOUTH BEACH CONDO at 1445 16th St Ph -3 ...
8,PAST SALE,2024-06-20,Condo/Co-op,1000 Brickell Plz Ph 5504,Miami,FL,33131.0,3550000.0,4.0,4.5,BRICKELL FLATIRON CONDO,2641.0,NaN,2019.0,NaN,1344.0,3683.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1000-Brickell-...,MARMLS,A11579153,N,Y,25.764264,-80.192725,9,blue,BRICKELL FLATIRON CONDO at 1000 Brickell Plz P...
9,PAST SALE,2024-06-20,Condo/Co-op,3917 N Meridian Ave #204,Miami Beach,FL,33140.0,3450000.0,1.0,2.0,THE WATERCLUB CONDO,709.0,NaN,1940.0,NaN,4866.0,550.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/3917-N-M...,MARMLS,A11556446,N,Y,25.812153,-80.133264,10,blue,THE WATERCLUB CONDO at 3917 N Meridian Ave #20...


## Time on Market Calculator

In [60]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2023, 11, 23) ## List (Earlier) date
date2 = datetime(2024, 6, 21) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

211
